## Getting the artist's exhibitions (announcements related to him)

In [2]:
import html
import requests
from bs4 import BeautifulSoup

artist = "A.K. Burns"
url = f"https://www.e-flux.com/announcements/?c[]=Contemporary%20Art&p[]={artist}" #Could replace %20 with a space

In [1]:
import scraping_functions

In [3]:
artist = "Hans Ulrich Obrist"
html = scraping_functions.get_html_of_artist(artist)
soup = BeautifulSoup(html, 'html.parser')

The HTML we get back:

In [167]:
i = 0
for line in soup:
    print(str(line)[:1000])
    i += 1
    if i > 2:
        break
print("...")

html


<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1" name="viewport"/>
<title>Announcements - e-flux</title>
<meta content="en" name="DC.LANGUAGE"/>
<link href="/styles/cbplayer.css?v=20220809025938" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/cblightbox.css?v=20210802045344" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/daterangepicker.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/main.css?v=20240226171903" media="all" rel="stylesheet" type="text/css"/>
<meta content="article" property="og:type"/>
<meta content="Announcements - e-flux" property="og:title"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Announcements - e-flux" name="twitter:title">
<meta co

--------------------------------------------------------------------------------------------

- We need to find the exhibitions from the code.<br>
Luckily, they seem to be stored all in the same format:

Example of what we're looking for in the HTML code:

```HTML

<a class="preview-announcement__title" href="/announcements/165278/lia-gangitano-to-receive-the-2018-audrey-irmas-award-for-curatorial-excellence/">
Lia Gangitano to receive the 2018 Audrey Irmas Award for Curatorial Excellence
</a>


or:

<a class="preview-announcement__title" href="/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/">
Rirkrit Tiravanija<br/><em>We Don’t Recognise What We Don’t See</em>
</a>
```

### **Get announcements: easy case**

In [3]:
announcements = scraping_functions.get_announcements_of_artist("Hans Ulrich Obrist") #May add: contemporary=True
scraping_functions.process_announcements(announcements)

[{'id': 507604,
  'link': '/announcements/507604/sin-wai-kindreaming-the-end/',
  'title_artists': 'Sin Wai Kin',
  'title': 'Dreaming the End'},
 {'id': 531326,
  'link': '/announcements/531326/suzanne-valadona-world-of-her-own/',
  'title_artists': 'Suzanne Valadon',
  'title': 'A World of Her Own'},
 {'id': 528799,
  'link': '/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/',
  'title_artists': 'Rirkrit Tiravanija',
  'title': 'We Don’t Recognise What We Don’t See'},
 {'id': 527878,
  'link': '/announcements/527878/young-climate-prize-2023-winners/',
  'title_artists': None,
  'title': 'Young Climate Prize 2023 winners'},
 {'id': 513946,
  'link': '/announcements/513946/50th-anniversary/',
  'title_artists': None,
  'title': '50th anniversary'},
 {'id': 508921,
  'link': '/announcements/508921/hoffnung-hoffnung/',
  'title_artists': None,
  'title': 'HOFFNUNG? HOFFNUNG!'},
 {'id': 505131,
  'link': '/announcements/505131/celebrating-15-years/',
  'title_

<details><summary><u>Special cases that we need to consider:</u></summary>

```Python
t = """
<a class="preview-announcement__title" href="/announcements/528799/rirkrit-tiravanijawe-don-t-recognise-what-we-don-t-see/">
Rirkrit Tiravanija<br/><em>We Don’t Recognise What We Don’t See</em>
</a>
"""
soup = BeautifulSoup(t, "html.parser")

# Find the 'em' tag
em_tag = soup.find('em')
# If the 'em' tag exists, extract its text
name = em_tag.get_text() if em_tag else None
```

### **Get announcements: hard case**

The problem is we can only gather maximum 30 exhibitions per artist this way, because the website only shows 30 exhibitions per artist if we don't scroll. We need to use something like Selenium to scroll the page and get all the exhibitions.

We can use Selenium (see the function file):

In [4]:
URL_Picasso = "https://www.e-flux.com/announcements/?p[]=Pablo%20Picasso"
results = scraping_functions.scroll_scrape_website(URL_Picasso)

In [5]:
for line in results:
    print(str(line)[:1000]) #Just to not have too much output printed
    print("...")
    break

<html class="no-touch cb-lightbox-no-touch" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, minimum-scale=1" name="viewport"/>
<title>Announcements - e-flux</title>
<meta content="en" name="DC.LANGUAGE"/>
<link href="/styles/cbplayer.css?v=20220809025938" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/cblightbox.css?v=20210802045344" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/daterangepicker.css" media="all" rel="stylesheet" type="text/css"/>
<link href="/styles/main.css?v=20240226171903" media="all" rel="stylesheet" type="text/css"/>
<meta content="article" property="og:type"/>
<meta content="Announcements - e-flux" property="og:title"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="Announcements - e-flux" 

In [6]:
announcements = (scraping_functions.get_announcements_html(results))
len(announcements)

177

Works with Picasso, we got 177 out of 177 exhibitions

<details><summary><u> Case when we want to click a "Load more" button (don't need it for artists' announcement scraping) </u></summary>

```python

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

# Go to your page url
url_Picasso = "https://www.e-flux.com/announcements/?p[]=Pablo%20Picasso"
driver.get(url_Picasso)

# Wait for the "Load more" button to become clickable
wait = WebDriverWait(driver, 10)

while True:
    try:
        # Wait until the "Load more" button is clickable, and then click it
        load_more_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Load more"]')))
        load_more_button.click()

        # Wait for the page to load
        time.sleep(2)
    except Exception as e:
        # If an error occurs, that probably means there's no more "Load more" button
        break

# Now you can parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Remember to close the driver
driver.quit()

```

In [7]:
announcements_picasso = scraping_functions.process_announcements(announcements)

In [8]:
announcements_picasso[0:10]

[{'id': 515809,
  'link': '/announcements/515809/picasso-sculptor-matter-and-body/',
  'title_artists': None,
  'title': 'Picasso Sculptor: Matter and Body'},
 {'id': 512737,
  'link': '/announcements/512737/chagall-matisse-mir-made-in-paris/',
  'title_artists': None,
  'title': 'Chagall, Matisse, Miró: Made in Paris'},
 {'id': 552019,
  'link': '/announcements/552019/picasso-ceramics/',
  'title_artists': None,
  'title': 'Picasso Ceramics'},
 {'id': 510210,
  'link': '/announcements/510210/jammie-holmesmake-the-revolution-irresistible/',
  'title_artists': 'Jammie Holmes',
  'title': 'Make the Revolution Irresistible'},
 {'id': 509314,
  'link': '/announcements/509314/outstanding/',
  'title_artists': None,
  'title': 'Outstanding!'},
 {'id': 530487,
  'link': '/announcements/530487/picasso-untitled/',
  'title_artists': None,
  'title': 'Picasso: Untitled'},
 {'id': 534375,
  'link': '/announcements/534375/call-for-applications-artistic-director/',
  'title_artists': None,
  'title

### Next step:

-Also collect announcement date, and some subtitle (these are not part of the announcement `<div>`, but separately, above the announcement's `div`)

## Collect exhibitions of all artists



Maybe for a first step, we collect the data for artists in `artists_contemporary_cleaned_v1.txt`? If we just get these types of exhibitions by them, then we only have to use Selenium twice:
- "Contemporary Art"
- "Data & Information"
- "Installation" 
- "Mixed Media"
- "Posthumanism"
- "Postmodernism"
- "Technology"

(I'd think these are mostly the contemporary categories on the website, feel free to add more). For these cases, only 2 artists have over 30 exhibitions in these categories: Hans Ulrich Obrist (36) and Olafur Eliasson (34). We can just use simple scraping for other artists.<br>
URL for these categories: https://www.e-flux.com/announcements/?c[]=Contemporary%20Art&c[]=Data%20%26%20Information&c[]=Installation&c[]=Mixed%20Media&c[]=Posthumanism&c[]=Postmodernism&c[]=Technology


In [ ]:
#TODO